# Different Optimization Methods For Quadratic Programming

$$
y=x^{\mathrm{T}}Wx \\
\frac{\partial y}{\partial x} = 2Wx
$$

Note that $W$ is positive semi-definite.

The condition number of matrix matters.

In [27]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [343]:
# W = np.array([[0.5, -0.2], [-0.2, 0.1]])
# condition number: 33.97056274847716
# W = np.array([[0.5, -0.2], [-0.2, 0.3]])
# condition number: 3.535322165454393
W = np.array([[4, 0.3], [0.3, 3]])
# condition number: 1.3998040576315456
eigs = np.linalg.eig(W)[0]
cond_num = max(eigs) / min(eigs)
print(cond_num)

1.3998040576315456


In [344]:
p = 50
x1s = np.linspace(-15, 15, p)
x2s = np.linspace(-15, 15, p)
x = np.stack(np.meshgrid(x1s, x2s), axis=-1)  # p x p x 2

def f(batch_x):
    # shape = (2, bsz)
    assert batch_x.shape[0] == 2
    return ((batch_x.T @ W) * batch_x.T).sum(-1)

y = f(x.reshape(-1, 2).T).reshape(p, p)

In [345]:
fig = make_subplots(rows=2, cols=2)

for (r, c) in [(1, 1), (1, 2), (2, 1), (2, 2)]:
    fig.add_trace(
        go.Contour(z=y, x=x1s, y=x2s), row=r, col=c
    )

# px = np.array([8, 8]).reshape(2, 1)
px = np.array([12, 2]).reshape(2, 1)

# conjugate gradient
r1 = np.array([1, 0]).reshape(2, 1)
alpha = - (px.T @ W @ r1) / (r1.T @ W @ r1)
px1 = px + alpha * r1

r2_ = (r1.T @ W).reshape(-1)
r2 = np.array([r2_[1], -r2_[0]]).reshape(2, 1)
alpha = - (px1.T @ W @ r2) / (r2.T @ W @ r2)
px2 = px1 + alpha * r2

pxs = [px, px1, px2]

fig.add_trace(
    go.Scatter(
        x=[ele[0][0] for ele in pxs], 
        y=[ele[1][0] for ele in pxs], 
    ),
    row=1, col=1)

# gradient descent
pxs = [px]
last_px = px
for _ in range(1, 31):
    r = 2 * W @ last_px
    last_px = last_px - r * 0.1
    pxs.append(last_px)

fig.add_trace(
    go.Scatter(
        x=[ele[0][0] for ele in pxs], 
        y=[ele[1][0] for ele in pxs], 
    ),
    row=1, col=2)

# gradient descent with momentum
beta = 0.95
pxs = [px]
mom = 0.0
last_px = px
for i in range(1, 31):
    r = 2 * W @ last_px
    # mom = (beta * mom + (1 - beta) * r) / (1 - beta ** i)
    mom = (beta * mom + (1 - beta) * r)
    last_px = last_px - mom * 0.1
    pxs.append(last_px)
fig.add_trace(
    go.Scatter(
        x=[ele[0][0] for ele in pxs], 
        y=[ele[1][0] for ele in pxs], 
    ),
    row=2, col=1)

# adam
beta1 = 0.9
beta2 = 0.95
mom1 = 0.0
mom2 = 0.0
pxs = [px]
last_px = px
for i in range(1, 31):
    r = 2 * W @ last_px
    mom1 = (beta1 * mom1 + (1 - beta1) * r) / (1 - beta1 ** i)
    mom2 = (beta2 * mom2 + (1 - beta2) * (r ** 2)) / (1 - beta2 ** i)
    last_px = last_px - mom1 / (mom2 ** 0.5 + 0.0000001) * 0.1
    pxs.append(last_px)
fig.add_trace(
    go.Scatter(
        x=[ele[0][0] for ele in pxs], 
        y=[ele[1][0] for ele in pxs], 
    ),
    row=2, col=2)

fig.update_layout(width = 600, height = 600,)
fig.update_xaxes(matches='x')
fig.update_yaxes(scaleanchor="x", scaleratio=1)
